In [2]:
import os
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
#file_path = "work/pi_wenlongzhao_umass_edu/8/tej/696-detecting-salient-entities/data/article_info.json"
# file_path = "./data/article_info.json"

In [ ]:
# with open(file_path, "r") as f:
#     data = json.load(f)
# df = pd.DataFrame(data)

In [ ]:
# model_paths = ['/datasets/ai/llama3/meta-llama/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6','datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa','datasets/ai/t5/models--google--t5-v1_1-base/snapshots/b5fc947a416ea3cb079532cb3c2bbadeb7f800fc']

In [2]:
# model_path = '/datasets/ai/llama3/hub/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6'

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")  # Move model to CUDA

# # Tokenize the prompt and move input to CUDA
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# # Generate response
# output_ids = model.generate(input_ids, max_length=1000)

# # Decode and print the response
# response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print(response)

In [ ]:
# quantization_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype="torch.float16",
# )

In [5]:
model_path = "/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map='auto')

2025-03-05 00:54:06.810773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741136046.826714 3892571 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741136046.831372 3892571 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 00:54:06.849326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/aranade_umass_edu/.conda/envs/intro/lib/python3.9/site-packages/accelerate/utils/modeling.py:1536: Us

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
title = 'FAA: Jets at New York airport directed by child'

article = '''A US air traffic controller and his supervisor have been placed on administrative leave and are under investigation after a small boy was apparently allowed
to direct passenger airplanes at New York's John F. Kennedy International Airport.A probe was started after tape recordings indicated that a small child, which The 
Telegraph describes as sounding seven or eight years old, was heard directing air traffic from the airport on February 16.JetBlue 171 contact departure, was one of the 
child's directions. The pilot of the addressed aircraft responded by saying: Over to departure JetBlue 171, awesome job. In another direction, the child says MS 4-0-3, 
contact departure [...] Adios, amigo. The pilot replied with Adios, amigo.A man is later heard saying [h]ere's what you get, guys, when the kids are out of school. The 
age of the child and the name of their father wasn't immediately available.An aviation official who asked to remain anonymous later told media that the same controller 
took his daughter into the control tower the next day, and that she was allowed to communicate with two pilots. According to another official, the two children are 
twins.The Federal Aviation Administration (FAA), meanwhile, announced that all unofficial visits to towers, radar rooms, and other air traffic control areas, would be 
prohibited while the investigation is being conducted. Pending the outcome of our investigation, the employees involved in this incident are not controlling air traffic,
the aviation authority said in a statement.Administrator Randy Babbitt commented about the incident. This lapse in judgment not only violated FAA's own policies but 
common sense standards for professional conduct. These kinds of distractions are totally unacceptable. We have an incredible team of professionals who safely control 
our nation's skies every single day. This kind of behavior does not reflect the true caliber of our work force.The National Air Traffic Controllers Association, which 
is the union representing controllers, also stated that [w]e do not condone this type of behaviour in any way. It is not indicative of the highest professional standards
that controllers set for themselves and exceed each and every day in the advancement of aviation safety.However, Dave Pascoe, the owner of LiveATC.net, a website which
keeps records of air traffic communications, including those involving the aforementioned child, described the incident as being blown out of proportion, saying to CNN 
that [...] when you listen to any of the recordings, the situation in the tower is very controlled. There is no hint [...] that anyone was too busy or anyone was 
interrupting the planes. [...] It was very controlled, and I don't think safety was compromised, nor should anyone be disciplined for this.[...] I have every belief that 
they'd make sure there were additional eyes there. JFK is highly supervised. It's not just one controller controlling the runway. Supervisors are there, and multiple 
people are there making sure by looking through binoculars and at radar. Pascoe is also a pilot."
'''

## Zero-shot prompt

In [9]:
# prompt = f'''
# Article Title: {title}
# Article Text: {article}

# You are an expert news article analysis assistant. The salience of an entity provides information about the importance or the centrality of that entity to the 
# article text. Given an article's title and its text, your task is to extract the entities mentioned in the article and assign a salience score to each entity. 
# The salience score is defined as follows:
# - 1: The entity is salient as per the article's context.
# - 0: The entity is mentioned but is not salient.

# Follow these rules:
# 1. Only extract entities that are explicitly mentioned in the article.
# 2. Do not include any entities that do not appear in the article.
# 3. Use only the provided title and text to determine salience.
# 4. Format your answer as valid JSON exactly as specified below.

# The expected JSON format is:
# {{
#   "entities": [
#     {{
#       "entity title": "<entity_name>",
#       "entity salience": "<0 or 1>"
#     }},
#     ...
#   ]
# }}

# Now, based on the article above, return only the final JSON output (with no extra commentary).
# '''

## Few-shot prompt with CoT

In [13]:
few_shot_CoT_prompt = f'''
Given an article's title and its text, your task is to extract the entities mentioned in the article and assign a salience score to each entity.

Below are two examples:

Example 1:
---------------------
Article Title: Laws allowing same sex marriage in Washington, D.C. go into effect
Article Text: The United States capital of Washington, D.C. legalized same-sex marriage on Wednesday. Beginning at 6 A.M. local time (1100 UTC), couples began submitting marriage applications at local courthouses citywide. Washington D.C. becomes the seventh United States territory to legalize same sex marriage. The bill was ratified by Mayor Adrian Fenty last December. Due to city's territorial status as a federal district, the bill had to be reviewed by congress. The bill passed congressional review Tuesday night. The bill faced opposition from many family values activists, who tried to stop the bill from becoming law. Supreme Court Chief Justice John Roberts rejected a lawsuit to prevent the measure.
Final JSON Output:
{{
  "entities": [
    {{
      "entity title": "United States",
      "entity salience": "1"
    }},
    {{
      "entity title": "Washington, D.C.",
      "entity salience": "1"
    }},
    {{
      "entity title": "same-sex marriage",
      "entity salience": "0"
    }},
    {{
      "entity title": "Adrian Fenty",
      "entity salience": "0"
    }},
    {{
      "entity title": "federal district",
      "entity salience": "0"
    }},
    {{
      "entity title": "family values",
      "entity salience": "0"
    }},
    {{
      "entity title": "John Roberts",
      "entity salience": "0"
    }}
  ]
}}
---------------------

Example 2:
---------------------
Article Title: New York Representative Eric Massa to retire
Article Text: New York Representative Eric Massa announced Wednesday that he would be stepping down as Congressman from New York's 29th congressional district. He cited health reasons for the sudden announcement. Massa is the latest in a string of United States Congresspeople to resign or not to seek reelection in 2010. He said that he had his third non-specific cancer recurrence in December 2009. He implied that his cancer is terminal, saying “I will now enter the final phase of my life at a more controlled pace.” He denied claims that his resignation is related to a sexual harassment accusation from a male aide. He said that this was untrue, although he admitted to using so called salty language.
Final JSON Output:
{{
  "entities": [
    {{
      "entity title": "New York",
      "entity salience": "1"
    }},
    {{
      "entity title": "Eric Massa",
      "entity salience": "0"
    }},
    {{
      "entity title": "Congressman",
      "entity salience": "1"
    }},
    {{
      "entity title": "New York",
      "entity salience": "1"
    }},
    {{
      "entity title": "29th congressional district",
      "entity salience": "0"
    }},
    {{
      "entity title": "United States",
      "entity salience": "1"
    }},
    {{
      "entity title": "sexual harassment",
      "entity salience": "0"
    }}
  ]
}}
---------------------

Now, use the following chain-of-thought (CoT) instructions:

Before producing the final JSON output, think through your reasoning for each extracted entity:
- Identify each entity mentioned explicitly in the article.
- For each entity, decide if it is central (salience "1") or merely mentioned (salience "0") based solely on the provided title and text.
- Explain your reasoning briefly in your mind (or as intermediate steps), but do not include these explanations in your final answer.

Finally, after your internal chain-of-thought reasoning, output only the final JSON output (with no extra commentary) exactly in the format below:

{{
  "entities": [
    {{
      "entity title": "<entity_name>",
      "entity salience": "<0 or 1>"
    }},
    ...
  ]
}}

Now, based on the article below, perform your reasoning and then return only the final JSON answer.

---------------------
Article Title: {title}
Article Text: {article}
---------------------
'''

In [15]:
messages = [
    {"role": "system", "content": "You are an expert news article analysis assistant. The salience of an entity provides information about the importance or the centrality of that entity to the article text."},
    {"role": "user", "content": few_shot_CoT_prompt}
]

In [19]:
model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [17]:
response

NameError: name 'response' is not defined

In [17]:
decoded_output = '''
{\n  "entities": [\n    {\n      "entity title": "US",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "John F. Kennedy International Airport",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "FAA",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "aviation authority",\n      "entity salience": "0"\n    },\n    {\n      "entity title": "New York",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "air traffic controller",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "child",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "pilots",\n      "entity salience": "1"\n    },\n    {\n      "entity title": "National Air Traffic Controllers Association",\n      "entity salience": "0"\n    },\n    {\n      "entity title": "LiveATC.net",\n      "entity salience": "0"\n    },\n    {\n      "entity title": "Dave Pascoe",\n      "entity salience": "0"\n    }
'''

json_start = decoded_output.find('{')
json_end = decoded_output.rfind('}') + 1  # +1 to include the "}" in the slice
# bt
# Slice the decoded output to get the final JSON
final_json_output = decoded_output[json_start:json_end].strip()

print(final_json_output)

{
  "entities": [
    {
      "entity title": "US",
      "entity salience": "1"
    },
    {
      "entity title": "John F. Kennedy International Airport",
      "entity salience": "1"
    },
    {
      "entity title": "FAA",
      "entity salience": "1"
    },
    {
      "entity title": "aviation authority",
      "entity salience": "0"
    },
    {
      "entity title": "New York",
      "entity salience": "1"
    },
    {
      "entity title": "air traffic controller",
      "entity salience": "1"
    },
    {
      "entity title": "child",
      "entity salience": "1"
    },
    {
      "entity title": "pilots",
      "entity salience": "1"
    },
    {
      "entity title": "National Air Traffic Controllers Association",
      "entity salience": "0"
    },
    {
      "entity title": "LiveATC.net",
      "entity salience": "0"
    },
    {
      "entity title": "Dave Pascoe",
      "entity salience": "0"
    }


In [ ]:
model_paths = ['/datasets/ai/llama3/hub/models--meta-llama--Llama-3.2-1B-Instruct/snapshots/9213176726f574b556790deb65791e0c5aa438b6',
               'datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa',
               'datasets/ai/t5/models--google--t5-v1_1-base/snapshots/b5fc947a416ea3cb079532cb3c2bbadeb7f800fc'
              ]

### Loop over entire dataset

In [ ]:
"""
    Cleans incomplete JSON response by:
    1. Extracting the valid JSON part.
    2. Removing incomplete entity objects.
    3. Returning a properly formatted JSON dictionary.
    4. Removes repeated entity titles in the dictionary.
"""
def clean_incomplete_json(response):
 
    match = re.search(r'\{.*\}', response, re.DOTALL)
    if not match:
        raise ValueError("No valid JSON-like content found in the response.")

    json_like_str = match.group(0)


    json_like_str = json_like_str.replace("'", '"')  # Ensure valid JSON quotes
    json_like_str = re.sub(r',\s*}', '}', json_like_str)  # Remove trailing commas before }
    json_like_str = re.sub(r',\s*\]', ']', json_like_str)  # Remove trailing commas before ]

   
    entities_match = re.search(r'"entities":\s*\[(.*)', json_like_str, re.DOTALL)
    if not entities_match:
        raise ValueError("No 'entities' key found in the response.")

    entities_str = entities_match.group(1).strip()

   
    entity_blocks = re.findall(r'\{[^{}]*\}', entities_str, re.DOTALL)

    valid_entities = []
    for block in entity_blocks:
        try:
            entity = json.loads(block)  # Attempt to parse each block
            valid_entities.append(entity)  # Keep valid entities
        except json.JSONDecodeError:
            print(f"Skipping invalid entity: {block}")  # Debugging output

   
    cleaned_response = {"entities": valid_entities}
     
    cleaned_response["entities"] = list({e["entity title"]: e for e in cleaned_response["entities"]}.values())

    return cleaned_response
def fuzzy_match(title, candidates, threshold=85):
    best_match, score = process.extractOne(title, candidates)
    return best_match if score >= threshold else None

def evaluate_salience(ground_truth, model_output, threshold=85):
    gt_dict = {}
    for item in ground_truth:
        title = item.get('entity title')
        salience = item.get('entity salience')

        if title:
            title = title.lower()
            try:
                gt_dict[title] = int(salience) if salience is not None else 0
            except ValueError:
                gt_dict[title] = 0

    pred_dict = {}
    for item in model_output:
        title = item.get('entity title')
        salience = item.get('entity salience')

        if title:
            title = title.lower()
            try:
                pred_dict[title] = int(salience) if salience is not None else 0
            except ValueError:
                pred_dict[title] = 0
        else:
            print("Weird data : ",item)

    # Apply fuzzy matching for better entity alignment
    updated_pred_dict = {}
    for pred_title in pred_dict:
        matched_title = fuzzy_match(pred_title, list(gt_dict.keys()), threshold)
        if matched_title:
            updated_pred_dict[matched_title] = pred_dict[pred_title]
        else:
            updated_pred_dict[pred_title] = pred_dict[pred_title]

    all_entities = set(gt_dict.keys()).union(set(updated_pred_dict.keys()))
    y_true = [gt_dict.get(entity, 0) for entity in all_entities]
    y_pred = [updated_pred_dict.get(entity, 0) for entity in all_entities]

    return y_true, y_pred

def evaluate_multiple_instances(ground_truths, model_outputs, threshold=85):
    all_y_true = []
    all_y_pred = []
    
    for gt, mo in zip(ground_truths, model_outputs):
        y_true, y_pred = evaluate_salience(gt, mo, threshold)
        all_y_true.extend(y_true)
        all_y_pred.extend(y_pred)
        
    precision = precision_score(all_y_true, all_y_pred, average='macro')
    recall = recall_score(all_y_true, all_y_pred, average='macro')
    f1 = f1_score(all_y_true, all_y_pred, average='macro')
    
    # Compute accuracy based on fuzzy-matched salient entities
    all_salient_correct = all(
        gt_dict.get(entity, 0) == 1 and updated_pred_dict.get(entity, 0) == 1
        for gt, mo in zip(ground_truths, model_outputs)
        for gt_dict, updated_pred_dict in [
            (
                {fuzzy_match(item['entity title'].lower(), [x['entity title'].lower() for x in mo], threshold): int(item['entity salience']) for item in gt},
                {item['entity title'].lower(): int(item['entity salience']) for item in mo}
            )
        ]
        for entity in gt_dict if gt_dict[entity] == 1
    )

    accuracy = 1.0 if all_salient_correct else 0.0
    
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [ ]:
outputs = []
articles = list(df['text'])
titles = list(df['title'])
prompt_template = '''
Article Title: {title}
Article Text: {article}

You are an expert news article analysis assistant. Given an article's title and its text, your task is to extract the entities 
mentioned in the article and assign a salience score to each entity. The salience score is defined as follows:
• 1: The entity is central to the article's content.
• 0: The entity is mentioned but is not central.

Follow these rules:
1. Only extract entities that are explicitly mentioned in the article.
2. Do not include any entities that do not appear in the article.
3. Use only the provided title and text to determine salience.
4. Format your answer as valid JSON exactly as specified below.
5. Do not infinitely loop on the same words for any of the entity titles.
6. Every entity title should appear only once.

The expected JSON format is:
{{
  "entities": [
    {{
      "entity title": "<entity_name>",
      "entity salience": "<0 or 1>"
    }},
    ...
  ]
}}

Now, based on the article above, return only the final JSON output (with no extra commentary).
'''


####RUNS ZERO SHOT FOR EVERY MODEL SPECIFIED IN MODEL PATHS###########
i = 0
n = 5 #num instances
for model_path in model_paths[:1]:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")  # Move model to CUDA
    for title,article in zip(titles[:n],articles[:n]):
        prompt = prompt_template.format(title=title, article=article)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        
        # Generate response
        output_ids = model.generate(input_ids, max_new_tokens=1000)

        # Decode and print the response
        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        
        response = response.split("Now, based on the article above, return only the final JSON output (with no extra commentary).")[-1]
        result = clean_incomplete_json(response)
        print("Iteration", i)
        i+=1
        outputs.append(result['entities'])

f = open('outputs.txt','w+')
f.write(str(outputs))
f.close()

metrics = evaluate_multiple_instances(df['entities'][:n],outputs)
print("METRICS",metrics)
print("\n")
print(outputs)